# Initialization

In [1]:
using CSV, DataFrames, JuMP, HiGHS, LinearAlgebra, DataStructures, Statistics
baltic_demand = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/Demand_Baltic.csv", delim='\t'))
ports = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/ports.csv", delim='\t'))
distances = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/dist_dense.csv", delim='\t'))
baltic_fleet = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/fleet_Baltic.csv", delim='\t'))
fleet_data = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/fleet_data.csv", delim='\t'))
;

In [2]:
commodities = []
for i in 1:size(baltic_demand, 1)
    push!(commodities, string(baltic_demand.Origin[i], "s", "|", baltic_demand.Destination[i], "t"))
end

### Services:

The following array contains manual generated services to which the column generation will be applied.

In [3]:
# services = [["Feeder_800" "DEBRV" "NOKRS" "DEBRV"],
#             ["Feeder_800" "RUKGD" "FIKTK" "RUKGD"],
#             ["Feeder_450" "DEBRV" "NOSVG" "DEBRV"],
#             ["Feeder_450" "NOAES" "NOBGO" "NOAES"],
#             ["Feeder_450" "NOKRS" "SEGOT" "DKAAR" "NOKRS"],
#             ["Feeder_800" "DEBRV" "SEGOT" "DKAAR" "DEBRV"],
#             ["Feeder_450" "DEBRV" "NOSVG" "NOBGO" "NOAES" "DEBRV"],
#             ["Feeder_450" "SEGOT" "DKAAR" "PLGDY" "RUKGD" "SEGOT"],
#             ["Feeder_450" "DEBRV" "NOAES" "DEBRV" "RULED" "DEBRV"],
#             ["Feeder_450" "DEBRV" "RUKGD" "PLGDY" "DKAAR" "SEGOT" "DEBRV"],
#             ["Feeder_800" "PLGDY" "RUKGD" "RULED" "FIKTK" "FIRAU" "PLGDY"],
#             ["Feeder_450" "PLGDY" "RUKGD" "FIKTK" "RULED" "FIRAU" "PLGDY"],
#             ["Feeder_450" "NOKRS" "SEGOT" "DKAAR" "PLGDY" "RUKGD" "PLGDY" "DKAAR" "SEGOT" "NOKRS"],
#             ["Feeder_800" "NOAES" "NOBGO" "NOSVG" "NOKRS" "SEGOT" "DKAAR" "SEGOT" "NOKRS" "NOSVG" "NOBGO" "NOAES"]]

# services = [[33.0 2 "Feeder_800" "DEBRV" "NOKRS" "DEBRV"],
#                 [33.0 2 "Feeder_800" "NOKRS" "NOSVG" "NOBGO" "NOAES" "NOKRS"],
#                 [33.0 2 "Feeder_450" "NOKRS" "SEGOT" "DKAAR" "NOKRS"],
#                 [33.0 2 "Feeder_800" "DKAAR" "PLGDY" "RUKGD" "DKAAR"],
#                 [33.0 2 "Feeder_450" "SEGOT" "FIRAU" "FIKTK" "RULED" "SEGOT"]]

# services = [[33.0 2 "Feeder" "DEBRV" "DKAAR" "SEGOT" "FIRAU" "DEBRV"],
#             [33.0 2 "Feeder" "NOKRS" "DKAAR" "SEGOT" "NOBGO" "NOKRS"]]

services = [
    [110000 1 "Feeder_450" "DEBRV" "NOSVG" "NOBGO" "NOAES" "DEBRV"],
    [100000 1 "Feeder_450" "DEBRV" "RUKGD" "FIKTK" "FIRAU" "PLGDY" "DEBRV"],
    [90000 1 "Feeder_450" "DEBRV" "DKAAR" "SEGOT" "NOKRS" "DEBRV"],
    [970000 1 "Feeder_800" "DEBRV" "SEGOT" "RULED" "DEBRV"]
]


# Initialize an empty Set to store unique ports
service_ports = []
# Iterate through the subarrays and add the unique ports to the set
for service in services
    for port in service[4:end]
        push!(service_ports, port)
    end
end
service_ports = unique(service_ports);


In [4]:
# List of ports that have intersecting service
intersect_ports = []
for i in 1:length(services)
    for j in i+1:length(services)
        push!(intersect_ports, intersect([[row[4:end]...] for row in services][i], [[row[4:end]...] for row in services][j]))
    end
end
intersect_ports = collect(Set(i for j in intersect_ports for i in j)); # Collecting unique transshipment nodes

# Adding transshipment nodes where services intersect
services_w_transshipment = []
transshipment_nodes = []
i = 1
for service in services
    updated_service = []
    for port in service
        if port in intersect_ports
            push!(updated_service, port * string(i)) # vi behøves i realiteten ikke at update vores serivces
            push!(transshipment_nodes, port * string(i))
        else
            push!(updated_service, port)
        end
    end
    push!(services_w_transshipment, updated_service)
    i += 1
end
transshipment_nodes = unique(transshipment_nodes);

In [5]:
# Source and terminal adjacency list
s_t_adj = []
for port in service_ports
    for node in transshipment_nodes
        if port == node[1:5]
            push!(s_t_adj, [string(port, "s"), node, parse.(Float64, ports[ports.UNLocode .== port, 9][1])  + ports[ports.UNLocode .== port, 12][1]])
            push!(s_t_adj, [node, string(port, "t"), parse.(Float64, ports[ports.UNLocode .== port, 9][1]) + ports[ports.UNLocode .== port, 12][1]])
        end
    end
    if port ∉ intersect_ports
        push!(s_t_adj, [string(port, "s"), port, parse.(Float64, ports[ports.UNLocode .== port, 9][1])])
        push!(s_t_adj, [port, string(port, "t"), parse.(Float64, ports[ports.UNLocode .== port, 9][1])])
    end
end

# Transshipment nodes adjacency list
transshipment_adj = []
for node in intersect_ports, i in transshipment_nodes, j in transshipment_nodes
    if j!=i && contains(i, node) && contains(j, node)
        push!(transshipment_adj, [i, j, parse.(Float64, ports[ports.UNLocode .== node, 10][1])]) # Skal vi også have move omkostninger med, da vi skal load og disch?
    end
end

# Service ports adjacency list
ports_adj = []
test = []
for service in 1:length(services_w_transshipment)
    for port in 5:length(services_w_transshipment[service]) # Hvad gør vi med port call omkostninger?
        push!(ports_adj, [services_w_transshipment[service][port-1], services_w_transshipment[service][port], 0.0]) # ports[ports.UNLocode .== services_w_transshipment[service][port][1:5], 12][1]]
    end
end

# Penalty edges adjacency list
penalty_adj = []
for k in 1:size(baltic_demand,1)
    push!(penalty_adj, [string(baltic_demand.Origin[k], "s"), string(baltic_demand.Destination[k], "t"), 1000])
end

dummy_adj = []
for port in 1:length(service_ports)
    push!(dummy_adj, [string(service_ports[port], "t"), nothing, 0.0])
end

edges = vcat(s_t_adj, transshipment_adj, ports_adj, penalty_adj, dummy_adj);

In [6]:
function get_G(edges)
    # Create an empty adjacency list
    graph = Dict{String, Vector{Tuple{Any, Float64}}}()  # Use Tuple to store (target, VC)
    #graph = Dict{String, Vector{Tuple{String, Float64}}}()  # Use Tuple to store (target, VC)

    # Populate the adjacency list
    for edge in edges
        source, target, weight = edge
        if haskey(graph, source)
            push!(graph[source], (target, weight))
        else
            graph[source] = [(target, weight)]
        end
    end

    return graph
end

get_G (generic function with 1 method)

In [7]:
function dijkstra(G, source, destination) # HUSK DUAL VARIABLES
    dist = Dict{Any, Float64}()
    prev = Dict{Any, Any}()
    Q = Set(keys(G))

    for v in Q
        dist[v] = Inf
        prev[v] = nothing
    end

    dist[source] = 0.0

    while !isempty(Q)
        min_value = Inf
        min_key = string()
        for i in Q
            if dist[i] <= min_value
                min_value = dist[i]
                min_key = i
            end
        end
        u = min_key

        if dist[u] == Inf
            break
        end
        if u == destination
            break
        end
        
        delete!(Q, u)

        for v in G[u]
            if haskey(G, v[1])
                alt = dist[u] + v[2]
                if alt < dist[v[1]]
                    dist[v[1]] = alt
                    prev[v[1]] = u
                end
            else
                break
            end
        end
    end

    S = []
    u = destination
    push!(S, u)

    while prev[u] != nothing
        push!(S, u)
        u = prev[u]
    end
    push!(S, source)
    
    return reverse(S[2:end]), dist[destination]
end


dijkstra (generic function with 1 method)

## Flowproblem

### Initialization

In [8]:
current_paths = []
current_costs = []
current_commodities = []
;

## Basis

In [9]:
# Bases med penalty edges
for i in 1:length(penalty_adj)
    push!(current_paths, penalty_adj[i][1:2])
    push!(current_costs, penalty_adj[i][3])
    append!(current_commodities, i)
end

# Initializing parameters
n_k = size(baltic_demand.Origin, 1) # Number of commodities
K = 1:n_k

n_s = size(services_w_transshipment, 1) # Number of services
S = 1:n_s

n_ff = size(penalty_adj, 1) # Number of penalty edges
ff = 1:n_ff
index_ff = 1:n_k # Index of ff paths - Could be hard coded

n_p = size(current_paths, 1) # Number of paths
P = 1:n_p
index_p = n_k+1:n_p  # Index of non-ff paths - Could be hardcoded

n_mv = size(baltic_fleet, 1)
v = 1:n_mv
;

In [10]:
# Initial base including penalty edges
PK = Matrix{Float64}(undef, n_k, 0) # Type of commodity for each path
mvs = Matrix{Float64}(undef, n_mv, 0) # Type of vessel on each commodity

for i in P
    pk = zeros(Float64, n_k)
    pk[findall(x -> x == string(current_paths[i][1], "|", current_paths[i][end]), commodities)[1]] = 1
    PK = [PK pk]
end

for i in services
    mv = zeros(Float64, n_mv)
    mv[findall(x -> x == i[3], baltic_fleet."Vessel class")[1]] = 1
    mvs = [mvs mv]
end

# Edges with capacity constraint
i_j = []
for edge in edges
    if edge ∈ (penalty_adj ∪ dummy_adj)
        break
    end
    push!(i_j, string(edge[1], "|", edge[2]))
end

n_ij = size(i_j, 1) # Number of edges
ij = 1:n_ij # Range of edges
;

### Actual problem

In [11]:
flowModel = Model(HiGHS.Optimizer)
@variable(flowModel, x[P] >= 0)
@variable(flowModel, 0 <= y[S] <= 1)
@objective(flowModel, Min, sum(services_w_transshipment[i][1]*y[i] for i in S) + sum(current_costs[j]*x[j] for j in P))
@constraint(flowModel, Y_k[i=1:n_k], sum(PK[i, j] * x[j] for j in P) == baltic_demand.FFEPerWeek[i]) # Paths for commodity k ∈ K
@constraint(flowModel, M_v[i=1:n_mv], sum(mvs[i, j] * y[j] for j in S) <= baltic_fleet.Quantity[i])
#print(flowModel)
optimize!(flowModel)
#solution_summary(flowModel)
;

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-24); columns 0(-26); elements 0(-26) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  4.9040000000e+06
HiGHS run time      :          0.00


In [12]:
# Initial paths including shortest path for all commodities and activated penalty edges
aij = Matrix{Float64}(undef, n_ij, 0) # 1 if path p for commodity k uses edge (i,j), else 0
uij = Matrix{Float64}(undef, n_ij, 0) # vessel capacity if service s uses edge (i,j), else 0
PK = Matrix{Float64}(undef, n_k, 0) # Type of commodity for each path

for i in 1:length(services_w_transshipment)
    us = zeros(Float64, n_ij)
    for j in 5:length(services_w_transshipment[i])
        us[findall(x -> x == string(services_w_transshipment[i][j-1], "|", services_w_transshipment[i][j]), i_j)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
        us[findall(x -> x == string(services_w_transshipment[i][j-1], "|", services_w_transshipment[i][j-1][1:5], "t"), i_j)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
        us[findall(x -> x == string(services_w_transshipment[i][j-1][1:5], "s", "|", services_w_transshipment[i][j-1]), i_j)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
    end
    uij = [uij us]
end

for i in 1:length(current_paths)
    pk = zeros(Float64, n_k)
    pk[findall(x -> x == string(current_paths[i][1], "|", current_paths[i][end]), commodities)[1]] = 1
    PK = [PK pk]
end

dual_Y_k = [dual(flowModel[:Y_k][i]) for i in K] # Dual variables for commodities at iteration 1 
dual_Y_ij = zeros(Float64, length(edges)) # Dual variables for edges at iteration 1 
reduced_cost = [-1.0]

    

1-element Vector{Float64}:
 -1.0

In [13]:
any(reduced_cost .< 0)

true

In [40]:

#while any(reduced_cost .< 0)
# Initializing feasible paths
candidate_paths = []
candidate_costs = []
candidate_commodities = []

# Setting up new network with reduced cost
edges_w_dual = deepcopy(edges)
for i in 1:length(edges_w_dual)
    edges_w_dual[i][3] = edges_w_dual[i][3] - dual_Y_ij[i]
end

for i in 1:size(baltic_demand, 1)
    path, cost = dijkstra(get_G(edges_w_dual), string(baltic_demand.Origin[i],"s"), string(baltic_demand.Destination[i],"t"))
    push!(candidate_paths, path)
    push!(candidate_costs, cost)
    append!(candidate_commodities, i)
end

reduced_cost = candidate_costs - dual_Y_k
println(reduced_cost)

for i in 1:length(candidate_paths[reduced_cost .< 0])
    ak = zeros(Float64, n_ij)
    for j in 2:length(candidate_paths[reduced_cost .< 0][i])
        ak[findall(x -> x == string(candidate_paths[reduced_cost .< 0][i][j-1], "|", candidate_paths[reduced_cost .< 0][i][j]), i_j)[1]] = 1
    end
    aij = [aij ak]
    push!(current_paths, candidate_paths[reduced_cost .< 0][i])
    push!(current_costs, candidate_costs[reduced_cost .< 0][i])
    append!(current_commodities, findall(x -> x == string(candidate_paths[reduced_cost .< 0][i][1], "|", candidate_paths[reduced_cost .< 0][i][end]), commodities)[1])
end

for i in 1:length(candidate_paths[reduced_cost .< 0])
    pk = zeros(Float64, n_k)
    pk[findall(x -> x == string(candidate_paths[reduced_cost .< 0][i][1], "|", candidate_paths[reduced_cost .< 0][i][end]), commodities)[1]] = 1
    PK = [PK pk]
end

P = 1:length(current_paths)
A = 1:size(aij, 2)

flowModel = Model(HiGHS.Optimizer)
@variable(flowModel, x[P] >= 0)
@variable(flowModel, 0 <= y[S] <= 1)
@objective(flowModel, Min, sum(services_w_transshipment[i][1]*y[i] for i in S) + sum(current_costs[j]*x[j] for j in P))
@constraint(flowModel, Y_ij[i=1:n_ij], sum(aij[i, j] * x[j+n_ff] for j in A) <= sum(uij[i, j] * y[j] for j in S)) # Paths that uses edge ij
@constraint(flowModel, Y_k[i=1:n_k], sum(PK[i, j] * x[j] for j in P) == baltic_demand.FFEPerWeek[i]) # Paths for commodity k ∈ K
@constraint(flowModel, M_v[i=1:n_mv], sum(mvs[i, j] * y[j] for j in S) <= baltic_fleet.Quantity[i])
optimize!(flowModel)

dual_Y_k = [dual(flowModel[:Y_k][i]) for i in K]
dual_Y_ij = [dual(flowModel[:Y_ij][i]) for i in ij]
append!(dual_Y_ij, zeros(Float64, length(penalty_adj) + length(dummy_adj)))
println("hey")
#end




# primal_x = value.(x[P])
# primal_y = value.(y[S])



[-134.5, 0.0, 0.0, -97.5, 0.0, -141.44444444444446, -134.5, 0.0, -141.44444444444446, -141.44444444444446, -134.5, 0.0, -141.44444444444446, 0.0, -134.5, -169.0, 0.0, -134.5, 0.0, -56.94444444444446, 0.0, -103.0]
Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
64 rows, 49 cols, 205 nonzeros
42 rows, 30 cols, 120 nonzeros
42 rows, 30 cols, 120 nonzeros
Presolve : Reductions: rows 42(-44); columns 30(-124); elements 120(-746)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
         14     4.0583514444e+06 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 14
Objective value     :  4.0583514444e+06
HiGHS run time      :          0.00
hey

In [30]:
current_commodities[111:end]

14-element Vector{Any}:
 22
  1
  4
  6
  7
  9
 10
 11
 13
 15
 16
 18
 20
 22

In [35]:
i_j

62-element Vector{Any}:
 "DEBRVs|DEBRV1"
 "DEBRV1|DEBRVt"
 "DEBRVs|DEBRV2"
 "DEBRV2|DEBRVt"
 "DEBRVs|DEBRV3"
 "DEBRV3|DEBRVt"
 "DEBRVs|DEBRV4"
 "DEBRV4|DEBRVt"
 "NOSVGs|NOSVG"
 "NOSVG|NOSVGt"
 ⋮
 "FIRAU|PLGDY"
 "PLGDY|DEBRV2"
 "DEBRV3|DKAAR"
 "DKAAR|SEGOT3"
 "SEGOT3|NOKRS"
 "NOKRS|DEBRV3"
 "DEBRV4|SEGOT4"
 "SEGOT4|RULED"
 "RULED|DEBRV4"

Berfore the master problem is formulated, the $c_{y}$ constants are calculated which correspond to the fixed port costs (more might be added later).

Moreover, the sum of all paths must be equal to $d_{k}$, in order to fulfill the demand. However later on, a new expensive penalty edge is added which penalizes all the containers that are not delivered to the commodity port.

As the decision variables, $x_{i,j}$ represent the edges in the liner shipping network (eventually paths), the variables are initialized by pairing the neigbor ports in the services above. Thereby, the master problem is reduced to only respect the strict subset of all feasible patterns i.e. patterns using the generated services.

In [ ]:
# edges with capacity constraint
i_j = []
for edge in edges
    if edge ∈ (penalty_adj ∪ dummy_adj)
        break
    end
    push!(i_j, string(edge[1], "|", edge[2]))
end

# Paths and costs from shortest path
for i in 1:size(baltic_demand, 1)
    path, cost = dijkstra(get_G(edges), string(baltic_demand.Origin[i],"s"), string(baltic_demand.Destination[i],"t"))
    push!(current_paths, path)
    push!(current_costs, cost)
    append!(current_commodity, i)
end

# Initializing parameters
n_k = size(baltic_demand.Origin, 1) # Number of commodities
K = 1:n_k

n_s = size(services_w_transshipment, 1) # Number of services
S = 1:n_s

n_ff = size(penalty_adj, 1) # Number of penalty edges
ff = 1:n_ff
index_ff = 1:n_k # Index of ff paths - Could be hard coded

n_p = size(current_paths, 1) # Number of paths
P = 1:n_p
index_p = n_k+1:n_p  # Index of non-ff paths - Could be hardcoded

n_ij = size(i_j, 1) # Number of edges
ij = 1:n_ij # Range of edges

n_mv = size(baltic_fleet, 1)
v = 1:n_mv
;

In [ ]:
# Initial paths including shortest path for all commodities and activated penalty edges
aij = Matrix{Float64}(undef, n_ij, 0) # 1 if path p for commodity k uses edge (i,j), else 0
uij = Matrix{Float64}(undef, n_ij, 0) # vessel capacity if service s uses edge (i,j), else 0

for i in index_p
    ak = zeros(Float64, n_ij)
    for j in 2:length(paths[i])
        ak[findall(x -> x == string(paths[i][j-1], "|", paths[i][j]), i_j)[1]] = 1
    end
    aij = [aij ak]
end


for i in 1:length(services_w_transshipment)
    us = zeros(Float64, n_ij)
    for j in 5:length(services_w_transshipment[i])
        us[findall(x -> x == string(services_w_transshipment[i][j-1], "|", services_w_transshipment[i][j]), i_j)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
        us[findall(x -> x == string(services_w_transshipment[i][j-1], "|", services_w_transshipment[i][j-1][1:5], "t"), i_j)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
        us[findall(x -> x == string(services_w_transshipment[i][j-1][1:5], "s", "|", services_w_transshipment[i][j-1]), i_j)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
    end
    uij = [uij us]
end



In [ ]:
flowModel = Model(HiGHS.Optimizer)
@variable(flowModel, x[P] >= 0)
@variable(flowModel, 0 <= y[S] <= 1)
@objective(flowModel, Min, sum(services_w_transshipment[i][1]*y[i] for i in S) + sum(costs[j]*x[j] for j in P))
#@constraint(flowModel, Y_ij[i=1:n_ij], sum(aij[i, j] * x[j+n_ff] for j in K) <= sum(uij[i, j] * y[j] for j in S)) # Paths that uses edge ij
@constraint(flowModel, Y_k[i=1:n_k], sum(PK[i, j] * x[j] for j in P) == baltic_demand.FFEPerWeek[i]) # Paths for commodity k ∈ K
@constraint(flowModel, M_v[i=1:n_mv], sum(mvs[i, j] * y[j] for j in S) <= baltic_fleet.Quantity[i])
print(flowModel)
optimize!(flowModel)
#solution_summary(flowModel)
;